In [1]:
!df -h

Filesystem                                     Size    Used   Avail Capacity iused ifree %iused  Mounted on
/dev/disk3s1s1                                926Gi    10Gi   802Gi     2%    425k  4.3G    0%   /
devfs                                         205Ki   205Ki     0Bi   100%     708     0  100%   /dev
/dev/disk3s6                                  926Gi    16Gi   802Gi     2%      16  8.4G    0%   /System/Volumes/VM
/dev/disk3s2                                  926Gi   6.6Gi   802Gi     1%    1.3k  8.4G    0%   /System/Volumes/Preboot
/dev/disk3s4                                  926Gi   5.3Mi   802Gi     1%      58  8.4G    0%   /System/Volumes/Update
/dev/disk1s2                                  500Mi   6.0Mi   481Mi     2%       1  4.9M    0%   /System/Volumes/xarts
/dev/disk1s1                                  500Mi   5.4Mi   481Mi     2%      29  4.9M    0%   /System/Volumes/iSCPreboot
/dev/disk1s3                                  500Mi   2.4Mi   481Mi     1%      91  4.9M   

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
!pip install -U tiktoken

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install einops

In [ ]:
!python3 -V

In [ ]:
!pip install pytest 

In [ ]:
!pip install -U transformers bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [10]:
import os
os.environ['HF_HOME'] = '/System/Volumes/Data/Users/ron/Documents/llm_models/'

In [4]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [5]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3
    )

    return results

In [23]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [8]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
torch.random.manual_seed(0)

In [11]:
device = torch.device("mps") if torch.backends.mps.is_available() else 'cpu'
device

device(type='mps')

In [13]:
from huggingface_hub import login

In [15]:
login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# Use a pipeline as a high-level helper
#from transformers import pipeline

#pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

In [ ]:
#from transformers import BitsAndBytesConfig

In [16]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",device_map="auto")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [28]:
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use mps


In [29]:
def build_prompt(query, search_results):
    prompt_template = """
QUESTION: {question}

CONTEXT:
{context}

ANSWER:
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"{doc['question']}\n{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()


In [30]:
rag("I just discovered the course. Can I still join it?")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top

'Yes, you can still join the course.'

#